In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

detector = HandDetector(maxHands=1)
offset = 20
imgSize = 300
counter = 0
folder = "File/4"

while True:
    success, img = cap.read()
    if not success:
        print("Error: Could not read frame.")
        break
    
    hands, img = detector.findHands(img)
    
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        # Check if cropping indices are within the image dimensions
        x_start = max(0, x - offset)
        y_start = max(0, y - offset)
        x_end = min(img.shape[1], x + w + offset)
        y_end = min(img.shape[0], y + h + offset)

        imgCrop = img[y_start:y_end, x_start:x_end]
        
        # Check if imgCrop is empty
        if imgCrop.size == 0:
            print("Cropped image is empty. Skipping this frame.")
            continue
        
        imgCropShape = imgCrop.shape
        aspectRatio = h / w

        if aspectRatio > 1:  
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap: wCal + wGap] = imgResize

        else: 
            k = imgSize / w
            hCal = math.ceil(k * h)
            # Ensure hCal does not exceed imgSize
            hCal = min(hCal, imgSize)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap: hCal + hGap, :] = imgResize

        cv2.imshow('ImageCrop', imgCrop)
        cv2.imshow('ImageWhite', imgWhite)

    cv2.imshow('Image', img)
    key = cv2.waitKey(1)
    
    if key == ord("s"):
        counter += 1
        cv2.imwrite(f'{folder}/Image_{time.time()}.jpg', imgWhite)
        print(counter)
    elif key == ord("q"):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()